<a href="https://colab.research.google.com/github/revy1817/NLP_BG/blob/main/HomeWork_9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf

import numpy as np
import os
import time
from glob import glob

In [2]:
path_folder = '/content/drive/MyDrive/Colab Notebooks/Учеба/GeekBrains/Введение в обработку естественного языка/Less 9/TrumpSpeech/'

Возьмем датасет выступлений трампа, и сдаем нейросеть которая будет генерировать его политические спичи

In [3]:
text = ''
files_path = os.path.join(path_folder,'*.txt')

for path in glob(files_path):
  text_local = open(path, 'rb').read().decode(encoding='utf-8')
  text += ' ' + text_local
  # length of text is the number of characters in it
  print('Length of one file text: {} characters'.format(len(text_local)))

print('Length of text: {} characters'.format(len(text)))

Length of one file text: 49737 characters
Length of one file text: 58149 characters
Length of one file text: 60811 characters
Length of one file text: 63409 characters
Length of one file text: 64514 characters
Length of one file text: 38989 characters
Length of one file text: 76668 characters
Length of one file text: 57081 characters
Length of one file text: 50472 characters
Length of one file text: 52640 characters
Length of one file text: 87029 characters
Length of one file text: 63291 characters
Length of one file text: 36830 characters
Length of one file text: 61155 characters
Length of one file text: 58316 characters
Length of one file text: 74654 characters
Length of one file text: 67534 characters
Length of one file text: 49376 characters
Length of one file text: 49082 characters
Length of one file text: 78082 characters
Length of one file text: 63122 characters
Length of one file text: 89735 characters
Length of one file text: 37173 characters
Length of one file text: 14610 cha

In [4]:
# The unique characters in the file
vocab = sorted(set(text))
print('{} unique characters'.format(len(vocab)))

84 unique characters


In [5]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [ ]:
text_as_int, text, len(text_as_int), len(text)

In [7]:
# The maximum length sentence you want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)


sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

In [8]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [9]:
# Batch size
BATCH_SIZE = 64

BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

In [10]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [ ]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim,
                                  batch_input_shape=[batch_size, None]),
                                 
        tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),

        tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),

         tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
        
        tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
                                   
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

In [ ]:
model = build_model(
    vocab_size=len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)

In [ ]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 84) # (batch_size, sequence_length, vocab_size)


In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (64, None, 256)           21504     
                                                                 
 lstm_4 (LSTM)               (64, None, 1024)          5246976   
                                                                 
 lstm_5 (LSTM)               (64, None, 1024)          8392704   
                                                                 
 lstm_6 (LSTM)               (64, None, 1024)          8392704   
                                                                 
 lstm_7 (LSTM)               (64, None, 1024)          8392704   
                                                                 
 dense_1 (Dense)             (64, None, 84)            86100     
                                                                 
Total params: 30,532,692
Trainable params: 30,532,692


In [ ]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 84)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.430464


In [ ]:
model.compile(optimizer='adam', loss=loss)

In [ ]:
!rm -rf ./training_checkpoints

In [ ]:
!ls ./training_checkpoints

ls: cannot access './training_checkpoints': No such file or directory


In [ ]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_freq=88*5,
    save_weights_only=True)

In [ ]:
EPOCHS = 60

In [ ]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/60
266/266 [==============================] - 99s 366ms/step - loss: 1.1693
Epoch 2/60
266/266 [==============================] - 100s 370ms/step - loss: 1.1196
Epoch 3/60
266/266 [==============================] - 99s 366ms/step - loss: 1.0788
Epoch 4/60
266/266 [==============================] - 101s 372ms/step - loss: 1.0457
Epoch 5/60
266/266 [==============================] - 100s 370ms/step - loss: 1.0152
Epoch 6/60
266/266 [==============================] - 99s 366ms/step - loss: 0.9885
Epoch 7/60
266/266 [==============================] - 100s 371ms/step - loss: 0.9640
Epoch 8/60
266/266 [==============================] - 99s 366ms/step - loss: 0.9393
Epoch 9/60
266/266 [==============================] - 100s 370ms/step - loss: 0.9162
Epoch 10/60
266/266 [==============================] - 100s 370ms/step - loss: 0.8930
Epoch 11/60
266/266 [==============================] - 99s 367ms/step - loss: 0.8723
Epoch 12/60
266/266 [==============================] - 100s 371ms/st

In [ ]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_60'

In [ ]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

In [ ]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (1, None, 256)            21504     
                                                                 
 lstm_8 (LSTM)               (1, None, 1024)           5246976   
                                                                 
 lstm_9 (LSTM)               (1, None, 1024)           8392704   
                                                                 
 lstm_10 (LSTM)              (1, None, 1024)           8392704   
                                                                 
 lstm_11 (LSTM)              (1, None, 1024)           8392704   
                                                                 
 dense_2 (Dense)             (1, None, 84)             86100     
                                                                 
Total params: 30,532,692
Trainable params: 30,532,692


In [ ]:
def generate_text(model, start_string):
    # Evaluation step (generating text using the learned model)

    # Number of characters to generate
    num_generate = 500

    # Converting our start string to numbers (vectorizing)
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    # Empty string to store our results
    text_generated = []

    # Low temperature results in more predictable text.
    # Higher temperature results in more surprising text.
    # Experiment to find the best setting.
    temperature = 1

    # Here batch size == 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)
        # using a categorical distribution to predict the character returned by the model
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()

        # Pass the predicted character as the next input to the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))

In [ ]:
# temperature = 0.1
text_ = generate_text(model, start_string=u"Biden win")
print(text_)

Biden wins. When we win, Pennsylvania wins and America wins. For decades our leaders put global interests and senators. I mean, would have been nice much easier, because here in Des Moines, Iowa. We stand on the shoulders of American heroes who crossed the ocean, blazed the trail, sailed the continent, tamed the wilderness, dug out the Panama canal, laid down the railroads, revolutionized industry, won two World Wars, defeated fascism and communism, and put a man on the face of the earth. And we are maki


Ответ получился так себе, после введеных слов сеть решила поставить точку и начать новое предложение на другую тему, попробуем изменить температуру на 1

In [ ]:
#1
text_ = generate_text(model, start_string=u"Biden wins")
print(text_)

Biden wins. I guess they had no choice and they have money, they go to end it. If I did it, and we worked hard, and it was like this, every place we go. Last night, you know where we were. You have an obligation to do it. So now they're all saying we win, I'll have to protect you." So they say, "Yeah," but he couldn't draw a crowd. So they put a circle, like he's so sad the wall are my great, he owns We Try, right? Not a mistake when he heart and somebody said, of our farmers can do it. I'm not doing it. 


Вот здесь получилось уже намного лучше, сеть логическую идею проталкивать от и почти до конца спича, если сильно вчитаться то видно логические ошибки, но в целом получилось хорошо

Попробуем давать разные тексты связанные с политкий, экономикой или соц вопросами, и посмотрим на результат

In [ ]:
text_ = generate_text(model, start_string=u"Federal investigation of Hunter Biden reaches critical")
print(text_)

Federal investigation of Hunter Biden reaches critical reasons because they don't want to say it. The radical left in the world and today, we will never ever … I promise to you, ride a breakup betrayears. That'll be very well. "I'm down number one terrorist, shirted neithen that trey put with our great congressmen, they're here, frankly. So there's other four score anyour family and deciding the fate of your country, because he actually had a historic numbers left gets how are we doing constantly sort of a hey did anything. Let's Democrat Phatharic


In [ ]:
text_ = generate_text(model, start_string=u"America is a nation that can be defined in a single word")
print(text_)

America is a nation that can be defined in a single word, not one of them. They get it. And what about this Kamala? How about his polls done? Remember when China gives her their farm inbertion. Protestors, judges and justices who interpret the constitution as written is because they didn't want to do it because it sounds luxury. But at some point you got to buy a cureful with the people that control Biden's plan, in just three and a half years under my administration, we've secured America's borders, fixed our broke and really he deserves another six


In [ ]:
text_ = generate_text(model, start_string=u"Biden is willing to destroy America in order to destroy Russia")
print(text_)

Biden is willing to destroy America in order to destroy Russia, Russia, Russia. That's the eace brother, what? We did the opposite. I have to say of a tremendous man. We will surprise, think of that. We protect our border, and that's what we're doing, and that's what we're doing, but we're rising rape contracted by these countries. With Japan, we've got negotiating with the newction laws here in America, we've done a fantastic job watching a new crime, right? And it was too late before the election. They were trying to be nice, for other r. I wonder why? A


In [ ]:
#1
text_ = generate_text(model, start_string=u"Black Lives Matter abbreviated BLM is a decentralized political and social movement that seeks to highlight racism, discrimination, and racial inequality experienced by black people ")
print(text_)

Black Lives Matter abbreviated BLM is a decentralized political and social movement that seeks to highlight racism, discrimination, and racial inequality experienced by black people that would destroyed Medicare as we keep on winning, Washington Democrats put the needs of illegal immigrants by the way. For that totall decide whether we will quickly return to record prosperity or whether we will always protect pre-existing conditions. America will land the first woman on the moon and the United States will be totally sick, don't worry about it. Who would even think this happened- And how bad drug overwhelm with a lot of help from around the world at risk. And the only reason
